# Mobility and Storage Time Calculator

The following code takes processed and "cleaned" water masks from a specified working directory and performs a series of operations to calculate the: 1) area-based floodplain reworking timescales (TR), channel overlap decay timescales (TM),the reworking efficiency timescale (TM:TR), the channel migration timescale (TW), and distribution of channel areas (AW); 2) the sediment storage time distributions (tstor) using a probabilistic random walk framework; 3) the reach transit times (treach); 4) the total sediment transit time (ttot).

Author: James (Huck) Rees; PhD Student, UCSB Geography

Date: November 17, 2025

## Import packages

In [2]:
import os
import numpy as np
import pandas as pd
from natsort import natsorted
import glob as glob_module
import math
import geopandas as gpd
import ast

import re
import fiona
import rasterio
from rasterio.mask import mask
from rasterio import warp
from rasterio.warp import transform_geom, calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling
from pyproj import CRS, Geod

from scipy.stats import linregress
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

import geemap
import ee
from geopy.distance import geodesic
from collections import defaultdict

# Authenticate with Google Earth Engine
ee.Initialize()

## Initialize functions to calculate floodplain reworking timescale (Tr), channel width timescale (Tm), channel width timescale (Tw) and channel area (Aw) distributions from mobility sheets

In [3]:
def extract_year(filename):
    pattern = r".*_(\d{4})_DSWE_level_\d+_cleaned.tif"
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    return None

def get_utm_epsg(lon, lat):
    zone_number = int((lon + 180) / 6) + 1
    is_northern = lat >= 0
    return 32600 + zone_number if is_northern else 32700 + zone_number

def get_aw_dist(base_directory, output_directory, reach_range=None):
    os.makedirs(output_directory, exist_ok=True)
    reach_dirs = [d for d in os.listdir(base_directory) if d.startswith("reach_") and os.path.isdir(os.path.join(base_directory, d))]

    for reach_dir in reach_dirs:
        try:
            reach_number = int(reach_dir.split('_')[1])

            if isinstance(reach_range, int) and reach_number != reach_range:
                continue
            elif isinstance(reach_range, tuple) and not (reach_range[0] <= reach_number <= reach_range[1]):
                continue

            cleaned_dir = os.path.join(base_directory, reach_dir, "Cleaned")
            if not os.path.exists(cleaned_dir):
                print(f"Cleaned folder not found for Reach {reach_number}.")
                continue

            tif_files = [f for f in os.listdir(cleaned_dir) if f.endswith(".tif")]
            aw_values = []

            for tif_file in tif_files:
                with rasterio.open(os.path.join(cleaned_dir, tif_file)) as src:
                    data = src.read(1)
                    transform = src.transform
                    bounds = src.bounds
                    centroid_lon = (bounds.left + bounds.right) / 2
                    centroid_lat = (bounds.top + bounds.bottom) / 2
                    utm_epsg = get_utm_epsg(centroid_lon, centroid_lat)

                    dst_crs = CRS.from_epsg(utm_epsg)
                    transform_utm, width, height = calculate_default_transform(
                        src.crs, dst_crs, src.width, src.height, *src.bounds)

                    reprojected = np.empty((height, width), dtype=data.dtype)

                    reproject(
                        source=data,
                        destination=reprojected,
                        src_transform=transform,
                        src_crs=src.crs,
                        dst_transform=transform_utm,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest
                    )

                    pixel_area = abs(transform_utm.a * transform_utm.e)
                    wet_pixel_count = np.sum(reprojected == 1)
                    total_area_m2 = wet_pixel_count * pixel_area
                    aw_values.append(total_area_m2)

            output_df = pd.DataFrame({'A_w_m2': aw_values})
            output_csv = os.path.join(output_directory, f"Reach_{reach_number}_aw_dist.csv")
            output_df.to_csv(output_csv, index=False)

            print(f"Saved corrected A_w totals for Reach {reach_number} to {output_csv}")

        except Exception as e:
            print(f"Error processing reach folder {reach_dir}: {e}")

def load_rasters(directory):
    rasters = {}
    for filepath in os.listdir(directory):
        if filepath.endswith('.tif'):
            year = extract_year(filepath)
            if year is not None:
                with rasterio.open(os.path.join(directory, filepath)) as src:
                    data = src.read(1)
                    transform = src.transform
                    bounds = src.bounds
                    
                    # Get centroid and determine UTM zone
                    centroid_lon = (bounds.left + bounds.right) / 2
                    centroid_lat = (bounds.top + bounds.bottom) / 2
                    utm_epsg = get_utm_epsg(centroid_lon, centroid_lat)
                    
                    # Calculate UTM transform to get correct pixel area in m²
                    dst_crs = CRS.from_epsg(utm_epsg)
                    transform_utm, width, height = calculate_default_transform(
                        src.crs, dst_crs, src.width, src.height, *src.bounds)
                    
                    # Reproject the mask to UTM
                    reprojected = np.empty((height, width), dtype=data.dtype)
                    reproject(
                        source=data,
                        destination=reprojected,
                        src_transform=transform,
                        src_crs=src.crs,
                        dst_transform=transform_utm,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest
                    )
                    
                    # Now pixel_area is in square meters
                    pixel_area = abs(transform_utm.a * transform_utm.e)
                    
                    # Store the reprojected mask (not the original!)
                    rasters[year] = (reprojected == 1, pixel_area)
    return dict(sorted(rasters.items()))

def calculate_reworked_areas(rasters):
    delta_t_areas = defaultdict(list)
    years = sorted(rasters.keys())
    for i in range(len(years)):
        t1 = years[i]
        base_mask, pixel_area = rasters[t1]
        union_mask = np.copy(base_mask)
        for j in range(i + 1, len(years)):
            t2 = years[j]
            current_mask, _ = rasters[t2]
            union_mask = np.logical_or(union_mask, current_mask)
            reworked_pixels = np.sum(union_mask) - np.sum(base_mask)
            delta_t = t2 - t1
            reworked_area_km2 = (reworked_pixels * pixel_area) / 1e6
            delta_t_areas[delta_t].append(reworked_area_km2)
    return delta_t_areas

def calculate_overlap_areas(rasters):
    """
    Calculate the overlap area (A_m) between baseline and future channel positions.
    
    For each baseline year, calculates the intersection area with all future years,
    representing pixels that remain channelized between the two time points.
    
    Parameters:
        rasters (dict): Dictionary mapping years to tuples of (binary_mask, pixel_area)
    
    Returns:
        dict: Dictionary mapping delta_t values to lists of overlap areas in km²
    """
    delta_t_areas = defaultdict(list)
    years = sorted(rasters.keys())
    
    for i in range(len(years)):
        t1 = years[i]
        base_mask, pixel_area = rasters[t1]
        
        for j in range(i + 1, len(years)):
            t2 = years[j]
            current_mask, _ = rasters[t2]
            
            # Calculate intersection (overlap) between baseline and current mask
            overlap_mask = np.logical_and(base_mask, current_mask)
            overlap_pixels = np.sum(overlap_mask)
            
            delta_t = t2 - t1
            overlap_area_km2 = (overlap_pixels * pixel_area) / 1e6
            delta_t_areas[delta_t].append(overlap_area_km2)
    
    return delta_t_areas

def reworking_exponential(x, Pr_over_AW, Cr):
    return -Pr_over_AW * np.exp(-Cr * x) + Pr_over_AW

def overlap_exponential(x, Pm_over_AW, Cm):
    """
    Exponential model for channel overlap decay.
    
    A_M/AW = (1 - Pm_over_AW) * exp(-Cm * x) + Pm_over_AW
    
    Args:
        x: Time (years)
        Pm_over_AW: Asymptotic minimum of overlap (normalized by active width)
        Cm: Overlap decay rate (year^-1)
    
    Returns:
        Predicted overlap area normalized by active width
    """
    return (1 - Pm_over_AW) * np.exp(-Cm * x) + Pm_over_AW

def calculate_pswitch(Tm_over_Tr,  P10=0.57413, P90=2.21004):
    """
    Map Tm:Tr to pswitch using 10th-90th percentile range with physical bounds.
    
    Maps P10 (inefficient reworking) → 0.50 (high switching)
    Maps P90 (efficient reworking) → 0.05 (minimal but non-zero switching)
    
    Physical justification for pswitch >= 0.05:
    - All rivers experience some stochasticity (floods, bank failures, cutoffs)
    - Prevents numerical instabilities in Monte Carlo simulations
    - Consistent with observed behavior of highly efficient systems (Yukon ≈ 0.06)
    
    Args:
        Tm_over_Tr (float): Ratio of overlap decay timescale to floodplain reworking timescales
        P10 (float): 10th percentile of Tm:Tr in dataset (default 0.57413)
        P90 (float): 90th percentile of Tm:Tr in dataset (default 2.21004)
    
    Returns:
        float: Switching probability [0.05, 0.50]
    """
    # Linear mapping from [P10, P90] to [0.50, 0.05]
    pswitch = 0.50 - 0.45 * ((Tm_over_Tr - P10) / (P90 - P10))
    
    # Clamp to physical bounds
    pswitch = max(0.05, min(0.50, pswitch))
    
    return pswitch

def calculate_Tw(delta_ts, Pr_over_AW, Cr, subsample_n=20):
    """
    Calculate the channel width timescale (Tw) by subsampling 
    the Greenberg exponential fit and performing linear regression.
    
    This timescale represents the time required to rework one channel width (Aw)
    of floodplain, calculated from the linear approximation of the exponential
    reworking curve.
    
    Parameters:
        delta_ts (list or array): Time intervals from the reworked area data
        Pr_over_AW (float): Plateau parameter from Greenberg exponential fit
        Cr (float): Decay rate from Greenberg exponential fit (year^-1)
        subsample_n (int): Number of points to subsample for linear regression (default 20)
    
    Returns:
        float: Tw, the linear channel migration timescale (years)
    """
    # Define the range for subsampling
    min_dt = min(delta_ts)
    max_dt = max(delta_ts)
    
    # Subsample the fitted exponential curve
    subsample_delta_t = np.linspace(min_dt, max_dt, subsample_n)
    subsample_ar_aw = reworking_exponential(subsample_delta_t, Pr_over_AW, Cr)
    
    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(subsample_delta_t, subsample_ar_aw)
    
    # Calculate Tw as inverse of slope
    Tw = 1 / slope
    
    return Tw

def plot_mobility_fits(river_name, ds_order, delta_ts_rework, data_rework, Pr_over_AW, Cr, Tr,
                       delta_ts_overlap, data_overlap, Pm_over_AW, Cm, Tm, output_path,
                       show_timescale_lines=True, show_equations=False):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Calculate x-axis limits for reworking plot
    x_min_rework = (min(delta_ts_rework) // 5) * 5
    x_max_rework = ((max(delta_ts_rework) + 4) // 5) * 5
    
    # Plot 1: Reworked Area with box plots
    bp1 = ax1.boxplot(data_rework, positions=delta_ts_rework, widths=0.8, patch_artist=True)
    for patch in bp1['boxes']:
        patch.set_facecolor('lightblue')
    
    # TR timescale line
    if show_timescale_lines:
        ax1.plot([0, Tr], [0, 1], 'k--', linewidth=1.5, alpha=0.7, zorder=2)
    
    # Fitted curve for reworked area
    x_fit_rework = np.linspace(min(delta_ts_rework), max(delta_ts_rework), 200)
    y_fit_rework = reworking_exponential(x_fit_rework, Pr_over_AW, Cr)
    ax1.plot(x_fit_rework, y_fit_rework, 'r-', linewidth=2, zorder=3)
    
    # Add equation to plot
    if show_equations:
        equation_text_1 = f'$A_{{R}}/A_{{W}} = {Pr_over_AW:.3f}(1 - e^{{-{Cr:.4f}t}})$\n$T_{{R}} = {Tr:.2f}$ years'
        ax1.text(0.05, 0.95, equation_text_1, transform=ax1.transAxes, 
                fontsize=18, verticalalignment='top', 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax1.set_xlabel('Time interval (years)', fontsize=12)
    ax1.set_ylabel('$A_{R} / A_{W}$ (Reworked area : Wetted area)', fontsize=12)
    ax1.set_title(f'{river_name} reach {ds_order}: floodplain reworking', fontsize=12)
    tick_positions = np.arange(x_min_rework, x_max_rework + 1, 5)
    ax1.set_xticks(tick_positions)
    ax1.set_xticklabels(tick_positions)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(x_min_rework, x_max_rework + 1)
    ax1.set_ylim(0, Pr_over_AW)
    
    # Calculate x-axis limits for overlap plot
    x_min_overlap = (min(delta_ts_overlap) // 5) * 5
    x_max_overlap = ((max(delta_ts_overlap) + 4) // 5) * 5
    
    # Plot 2: Overlap Decay with box plots
    bp2 = ax2.boxplot(data_overlap, positions=delta_ts_overlap, widths=0.8, patch_artist=True)
    for patch in bp2['boxes']:
        patch.set_facecolor('lightgreen')
    
    # TM timescale line
    if show_timescale_lines:
        ax2.plot([0, Tm], [1, 0], 'k--', linewidth=1.5, alpha=0.7, zorder=2)
    
    # Fitted curve for overlap decay
    x_fit_overlap = np.linspace(min(delta_ts_overlap), max(delta_ts_overlap), 200)
    y_fit_overlap = overlap_exponential(x_fit_overlap, Pm_over_AW, Cm)
    ax2.plot(x_fit_overlap, y_fit_overlap, 'b-', linewidth=2, zorder=3)
    
    # Add equation to plot
    if show_equations:
        equation_text_2 = f'$A_{{M}}/A_{{W}} = {1-Pm_over_AW:.3f}e^{{-{Cm:.4f}t}} + {Pm_over_AW:.3f}$\n$T_{{M}} = {Tm:.2f}$ years'
        ax2.text(0.05, 0.95, equation_text_2, transform=ax2.transAxes, 
                fontsize=11, verticalalignment='top', 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax2.set_xlabel('Time interval (years)', fontsize=12)
    ax2.set_ylabel('$A_{M} / A_{W}$ (Overlap area : Wetted area)', fontsize=12)
    ax2.set_title(f'{river_name} reach {ds_order}: overlap decay', fontsize=12)
    tick_positions = np.arange(x_min_overlap, x_max_overlap + 1, 5)
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels(tick_positions)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(x_min_overlap, x_max_overlap + 1)
    ax2.set_ylim(Pm_over_AW, 1)
        
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved mobility plots to {output_path}")

def calculate_mobility(river_name, ds_order, working_directory):
    """
    Calculate mobility metrics (Tr, Tm, Tw, Cr, Cm) for a single reach.
    
    Parameters:
        river_name (str): Name of the river
        ds_order (int): Reach number to process
        working_directory (str): Base working directory path
    
    Returns:
        dict: Dictionary containing mobility metrics:
              - Tr_yr: Floodplain reworking timescale (years)
              - Tm_yr: Channel overlap timescale (years)
              - Tw_yr: Channel width timescale (years)
              - Cr_peryr: Reworking decay rate (year^-1)
              - Cm_peryr: Overlap decay rate (year^-1)
              - median_Aw_m2: Median active channel width (m²)
              - Pr_over_Aw: Reworking plateau parameter
              - Pm_over_Aw: Overlap plateau parameter
              - Pswitch: Switching probability for random walk model
    """
    # Define paths
    base_raster_dir = f"{working_directory}/RiverMapping/RiverMasks/{river_name}"
    reach_dir = f"reach_{ds_order}"
    full_path = os.path.join(base_raster_dir, reach_dir, "Cleaned")
    
    output_dir = f"{working_directory}/RiverMapping/Mobility/{river_name}"
    aw_output_dir = os.path.join(output_dir, "AW_distributions")
    plot_output_dir = os.path.join(output_dir, "Mobility_plots")
    os.makedirs(aw_output_dir, exist_ok=True)
    os.makedirs(plot_output_dir, exist_ok=True)
    
    # Generate AW distribution using get_aw_dist()
    get_aw_dist(base_raster_dir, aw_output_dir, reach_range=ds_order)
    
    # Load the generated AW distribution
    aw_csv_path = os.path.join(aw_output_dir, f"Reach_{ds_order}_aw_dist.csv")
    aw_distribution = pd.read_csv(aw_csv_path)
    
    # Calculate median Aw from the distribution (in m²)
    median_aw_m2 = aw_distribution['A_w_m2'].median()
    
    # Convert to km² for use in normalization calculations
    median_aw_km2 = median_aw_m2 / 1e6
    
    # Load rasters (now returns only rasters, no median_aw)
    rasters = load_rasters(full_path)
    
    # Calculate reworked areas
    delta_t_areas = calculate_reworked_areas(rasters)
    delta_ts_rework = sorted(delta_t_areas.keys())
    data_rework = [[val / median_aw_km2 for val in delta_t_areas[dt]] for dt in delta_ts_rework]
    medians_rework = [np.median(vals) for vals in data_rework]
    
    # Fit Greenberg exponential to reworked area data
    x_data_rework = np.array(delta_ts_rework)
    y_data_rework = np.array(medians_rework)
    initial_guess_rework = [max(y_data_rework), 0.1]
    popt_rework, _ = curve_fit(reworking_exponential, x_data_rework, y_data_rework, p0=initial_guess_rework)
    Pr_over_AW, Cr = popt_rework
    
    # Calculate TR
    Tr = (1 / Cr) * (1 / Pr_over_AW)
    
    # Calculate Tw (channel width timescale)
    Tw = calculate_Tw(delta_ts_rework, Pr_over_AW, Cr)
    
    # Calculate overlap areas
    delta_t_overlap = calculate_overlap_areas(rasters)
    delta_ts_overlap = sorted(delta_t_overlap.keys())
    data_overlap = [[val / median_aw_km2 for val in delta_t_overlap[dt]] for dt in delta_ts_overlap]
    medians_overlap = [np.median(vals) for vals in data_overlap]
    
    # Fit overlap exponential to overlap data
    x_data_overlap = np.array(delta_ts_overlap)
    y_data_overlap = np.array(medians_overlap)
    initial_guess_overlap = [max(y_data_overlap), 0.1]
    popt_overlap, _ = curve_fit(overlap_exponential, x_data_overlap, y_data_overlap, p0=initial_guess_overlap)
    Pm_over_AW, Cm = popt_overlap
    
    # Calculate Tm
    Tm = (1 / Cm) * (1 / (1 - Pm_over_AW))
    
    # Calculate floodplain reworking efficiency ratio
    Tm_over_Tr = Tm / Tr
    
    # Calculate pswitch
    pswitch = calculate_pswitch(Tm_over_Tr)
    
    # Create plots
    plot_path = os.path.join(plot_output_dir, f"Reach_{ds_order}_mobility_fits.png")
    plot_mobility_fits(river_name, ds_order, delta_ts_rework, data_rework, Pr_over_AW, Cr, Tr,
                       delta_ts_overlap, data_overlap, Pm_over_AW, Cm, Tm, plot_path)
    
    # Return results
    results = {
        'Tr_yr': Tr,
        'Tw_yr': Tw,
        'Tm_over_Tr': Tm_over_Tr,
        'Tm_yr': Tm,
        'Cr_peryr': Cr,
        'Cm_peryr': Cm,
        'median_Aw_m2': median_aw_m2,
        'Pr_over_Aw': Pr_over_AW,
        'Pm_over_Aw': Pm_over_AW,
        'Pswitch': pswitch
    }
    
    return results
    
def get_mobility(csv_path):
    """
    Iterates through a CSV file of river names and paths, calculating mobility metrics
    (Tr, Tm, Tw, Cr, Cm) for each reach as specified.
    
    Parameters:
        csv_path (str): Path to the CSV file with columns:
                        - river_name
                        - working_directory
                        - reach_range (e.g., "All", "(1, 3)", or "2")
    
    Outputs:
        CSV file: Mobility metrics saved to Mobility/{river_name}/{river_name}_mobility_metrics.csv
                  with columns: ds_order, Tr_yr, Tm_yr, Tw_yr, Cr_peryr, Cm_peryr, median_Aw_m2, Pr_over_Aw, Pm_over_Aw
    """
    river_data = pd.read_csv(csv_path)
    
    for _, row in river_data.iterrows():
        river_name = row['river_name']
        working_directory = row['working_directory']
        reach_range = row['reach_range']
        
        # Define base directory for raster masks
        base_raster_dir = f"{working_directory}/RiverMapping/RiverMasks/{river_name}"
        output_dir = f"{working_directory}/RiverMapping/Mobility/{river_name}"
        os.makedirs(output_dir, exist_ok=True)
        
        # Parse reach_range to determine which reaches to process
        reach_dirs = [d for d in os.listdir(base_raster_dir) if d.startswith("reach_")]
        available_reaches = sorted([int(d.split('_')[1]) for d in reach_dirs])
        
        if isinstance(reach_range, str):
            reach_range = reach_range.strip()
            if reach_range == "All":
                reaches_to_process = available_reaches
            elif reach_range.startswith("(") and reach_range.endswith(")"):
                # Parse tuple format like "(1, 3)"
                reach_start, reach_end = map(int, reach_range.strip("()").split(","))
                reaches_to_process = [r for r in available_reaches if reach_start <= r <= reach_end]
            else:
                # Single reach number
                reaches_to_process = [int(reach_range)]
        elif isinstance(reach_range, int):
            reaches_to_process = [reach_range]
        elif isinstance(reach_range, tuple):
            reach_start, reach_end = reach_range
            reaches_to_process = [r for r in available_reaches if reach_start <= r <= reach_end]
        else:
            raise ValueError(f"Invalid reach_range format: {reach_range}")
        
        # Calculate mobility metrics for each reach
        all_results = []
        
        for ds_order in reaches_to_process:
            try:
                print(f"Processing {river_name} Reach {ds_order}...")
                results = calculate_mobility(river_name, ds_order, working_directory)
                
                # Add ds_order to results
                results['ds_order'] = ds_order
                all_results.append(results)
                
            except Exception as e:
                print(f"Error processing {river_name} Reach {ds_order}: {e}")
                continue
        
        # Compile results into DataFrame
        if all_results:
            results_df = pd.DataFrame(all_results)
            
            # Reorder columns for clarity
            column_order = ['ds_order', 'Tr_yr', 'Tm_yr', 'Tm_over_Tr', 'Tw_yr', 'Cr_peryr', 'Cm_peryr', 'median_Aw_m2', 'Pr_over_Aw', 'Pm_over_Aw', 'Pswitch']
            results_df = results_df[column_order]
            
            # Sort by ds_order
            results_df = results_df.sort_values('ds_order')
            
            # Save to CSV
            output_csv = os.path.join(output_dir, f"{river_name}_mobility_metrics.csv")
            results_df.to_csv(output_csv, index=False)
            print(f"Saved mobility metrics for {river_name} to {output_csv}")
        else:
            print(f"No results generated for {river_name}")

## Initialize functions to calculate first-passage time distributions AKA sediment storage time distributions (Tstor)

In [4]:
def import_aw_distribution(river_name, reach_number, working_directory):
    """
    Imports the AW distribution for a specified reach.

    Args:
        river_name (str): Name of the river.
        reach_number (int): Reach number to import AW distribution.
        working_directory (str): Base working directory containing the river data.

    Returns:
        DataFrame: A DataFrame containing the AW distribution for the specified reach.
    """
    # Define base directory for AW distributions
    aw_dir = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, 'Aw_distributions')

    # Ensure the directory exists
    if not os.path.exists(aw_dir):
        raise FileNotFoundError(f"Aw distribution directory not found: {aw_dir}")

    # Load AW distribution for the specified reach
    aw_file = os.path.join(aw_dir, f"Reach_{reach_number}_aw_dist.csv")
    if not os.path.isfile(aw_file):
        raise FileNotFoundError(f"Aw file not found: {aw_file}")

    aw_distribution = pd.read_csv(aw_file)

    return aw_distribution

def calculate_tstor_distribution(channel_belt_area, aw_distribution, tw, pswitch, num_iterations=10000, max_timesteps=10000):
    """
    Calculates the Tstor distribution for a single reach using the random walk/single event model and returns the result.
    
    The channel undergoes a random walk, sampling Aw values from the distribution at each timestep,
    until it returns to its starting position x0. Reflection at boundaries automatically triggers direction switching.
    
    Args:
        channel_belt_area (float): Channel belt area (in square km) for the reach.
        aw_distribution (DataFrame): DataFrame with column 'A_w_m2' (in square m).
        tw (float): Lateral migration timescale.
        pswitch (float): The probability of the channel switching direction at each timestep tw.
        num_iterations (int): Number of simulations to run (default 10000).
        max_timesteps (int): Maximum timesteps per simulation (default 10000).
    
    Returns:
        DataFrame: A DataFrame containing the TFP distribution.
    """
    # Convert channel belt area from km² to m²
    channel_belt_area_m2 = channel_belt_area * 1_000_000
    
    if aw_distribution.empty:
        raise ValueError("Aw distribution is empty.")
    
    # Extract all AW values for random sampling
    aw_values = aw_distribution['A_w_m2'].values
    
    tfp_times = []
    
    for _ in range(num_iterations):
        x0 = np.random.uniform(0, channel_belt_area_m2)
        x = x0
        total_time = 0
        timestep_count = 0
        current_direction = np.random.choice([-1, 1])  # Initialize first direction randomly
        
        while timestep_count < max_timesteps:
            # Sample a new AW value for this timestep
            aw_step = np.random.choice(aw_values)
            
            # Determine direction based on switching probability BEFORE calculating x_intended
            if np.random.random() < pswitch:
                # Switch direction based on probability
                current_direction = -current_direction
            
            direction = current_direction
            x_intended = x + direction * aw_step
            
            reflection_occurred = False
            
            # Reflect at boundaries and FORCE direction switch on reflection
            if x_intended < 0:
                x_new = -x_intended
                reflection_occurred = True
                current_direction = -current_direction  # Automatic switch on reflection
            elif x_intended > channel_belt_area_m2:
                x_new = 2 * channel_belt_area_m2 - x_intended
                reflection_occurred = True
                current_direction = -current_direction  # Automatic switch on reflection
            else:
                x_new = x_intended
            
            # Check if we've crossed x0 (returned to starting position)
            crossed = False
            
            if reflection_occurred:
                if (x_intended < x0 and x_new >= x0) or (x_intended > x0 and x_new <= x0) or (timestep_count > 0 and x == x0 and x_new != x0):
                    crossed = True
            else:
                if (x_new >= x0 and x < x0) or (x_new <= x0 and x > x0):
                    crossed = True
            
            if crossed:
                # Calculate fractional time
                if reflection_occurred:
                    if x_intended < 0:
                        distance_to_boundary = abs(x - 0)
                        distance_from_boundary_to_x0 = abs(x0 - 0)
                        total_distance_to_x0 = distance_to_boundary + distance_from_boundary_to_x0
                    else:
                        distance_to_boundary = abs(channel_belt_area_m2 - x)
                        distance_from_boundary_to_x0 = abs(channel_belt_area_m2 - x0)
                        total_distance_to_x0 = distance_to_boundary + distance_from_boundary_to_x0
                    
                    fractional_tw = (total_distance_to_x0 / aw_step) * tw
                else:
                    remaining_distance = abs(x0 - x)
                    fractional_tw = (remaining_distance / aw_step) * tw
                
                total_time += fractional_tw
                break
            
            total_time += tw
            x = x_new
            timestep_count += 1
        
        if timestep_count < max_timesteps:
            tfp_times.append(total_time)
    
    result_df = pd.DataFrame({'Tstor_yr': tfp_times})
    
    return result_df

def get_tstor_distributions(csv_path):
    """
    Processes a range of reaches from a CSV file and calculates TFP distributions for each.

    Args:
        csv_path (str): Path to the CSV file containing river name, working directory, and reach range.

    Outputs:
        CSV files containing Tstor distributions for each processed reach.
    """
    # Load the configuration CSV
    config_data = pd.read_csv(csv_path)

    for index, row in config_data.iterrows():
        # Extract river name, reach range, and working directory for each row
        river_name = row['river_name']
        reach_range = row['reach_range']
        working_directory = row['working_directory']
        num_iterations = row['model_iterations']
        max_timesteps = row['max_timesteps']

        # Define directories for required inputs
        channel_belt_file = os.path.join(working_directory, 'ChannelBelts', 'Extracted_ChannelBelts', river_name, f"{river_name}_channelbelt_areas.csv")

        # Check if the required files exist
        if not os.path.isfile(channel_belt_file):
            raise FileNotFoundError(f"Channel belt areas file not found: {channel_belt_file}")
        
        # Load channel belt areas data
        channel_belt_data = pd.read_csv(channel_belt_file)

        # Load mobility metrics file once per river
        mobility_file = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, f"{river_name}_mobility_metrics.csv")
        if not os.path.isfile(mobility_file):
            raise FileNotFoundError(f"Mobility metrics file not found: {mobility_file}")
        mobility_all_reaches = pd.read_csv(mobility_file)

        # Determine the reach range
        if isinstance(reach_range, str):
            reach_range = reach_range.strip()  # Remove any extra spaces

            if reach_range == "All":
                reach_start = channel_belt_data['ds_order'].min()
                reach_end = channel_belt_data['ds_order'].max()
            elif reach_range.isdigit():
                # Convert a numeric string to an integer
                reach_range = int(reach_range)
                reach_start = reach_range
                reach_end = reach_range
            elif re.match(r'^\(\d{1,4}, \d{1,4}\)$', reach_range):  # Match (XX, YY) with 1 to 4 digits
                try:
                    # Convert the string to a tuple of integers
                    reach_range = ast.literal_eval(reach_range)
                    reach_start, reach_end = reach_range
                except (ValueError, SyntaxError):
                    raise ValueError(f"Invalid reach range format: {reach_range}")
            else:
                raise ValueError(f"Invalid string format for reach_range: {reach_range}")
        elif isinstance(reach_range, (int, float)) and float(reach_range).is_integer():
            # Convert float-like integers (e.g., 7.0) to int
            reach_range = int(reach_range)
            reach_start = reach_range
            reach_end = reach_range
        elif isinstance(reach_range, tuple) and len(reach_range) == 2:
            reach_start, reach_end = reach_range
        else:
            raise ValueError("reach_range must be 'All', an int, or a tuple (start, end).")

        # Generate range of reaches to process
        reaches = range(reach_start, reach_end + 1)

        # Iterate through the range of reaches and calculate Tstor for each
        for reach_number in reaches:
            # Get Tr and Cr values for this reach
            tw = mobility_all_reaches.loc[mobility_all_reaches['ds_order'] == reach_number, 'Tw_yr'].values[0]
            pswitch = mobility_all_reaches.loc[mobility_all_reaches['ds_order'] == reach_number, 'Pswitch'].values[0]

            # Get channel belt area for the reach
            channel_belt_area = channel_belt_data.loc[channel_belt_data['ds_order'] == reach_number, 'area_sq_km'].values[0]

            # Import AW distribution for the reach
            aw_distribution = import_aw_distribution(river_name, reach_number, working_directory)

            # Calculate the Tstor distribution for the reach
            tstor_distribution = calculate_tstor_distribution(channel_belt_area, aw_distribution, tw, pswitch, num_iterations, max_timesteps)

            # Save Tstor distribution to a CSV (fixed typo: sinlge -> single)
            output_file = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, 'Tstor_distributions', f"Reach_{reach_number}_Tstor_distribution.csv")
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            tstor_distribution.to_csv(output_file, index=False)

            print(f"Tstor distribution for Reach {reach_number} saved to {output_file}")

## Initialize functions to run Monte Carlo simulation to calculate reach transit time (t_reach) from the number of storage events (n) and storage time (tstor) distributions

In [5]:
def monte_carlo_reach_transit_time(
    tstor_df,
    transit_df,
    reach_number,
    num_iterations = 10000
    ):
    """
    Monte Carlo simulation of reach transit times using tstor sampling based on fractional 'n_stor'.
    
    Parameters:
        tstor_df (pd.DataFrame): One-column DataFrame of storage time values (e.g. 'Tstor_yr').
        transit_df (pd.DataFrame): DataFrame with 'ds_order' and 'n_stor' columns.
        reach_number (int): Reach number (ds_order) for simulation.
        num_iterations (int): Number of Monte Carlo simulations to run.
    Returns:
        pd.DataFrame: DataFrame of simulated transit times.
    """
    # Clean and check input
    transit_df.columns = transit_df.columns.str.strip()
    if "ds_order" not in transit_df or "n_stor" not in transit_df:
        raise KeyError("transit_df must contain 'ds_order' and 'n_stor' columns.")
    
    if tstor_df.shape[1] != 1:
        raise ValueError("tstor_df must contain exactly one column.")
    
    tstor_vals = tstor_df.iloc[:, 0].dropna().values
    if len(tstor_vals) == 0:
        raise ValueError("No valid storage time data found.")
    
    # Extract n_stor
    n_array = transit_df.loc[transit_df["ds_order"] == reach_number, "n_stor"].values
    if len(n_array) == 0:
        raise ValueError(f"Reach {reach_number} not found in transit_df.")
    
    n = float(n_array[0])
    int_part = int(np.floor(n))
    frac_part = n - int_part
    
    results = []
    for _ in range(num_iterations):
        # Handle the integer part
        if int_part > 0:
            samples = np.random.choice(tstor_vals, size=int_part, replace=True)
            total = samples.sum()
        else:
            total = 0.0
        
        # Handle the fractional part
        if np.random.rand() < frac_part:
            extra = np.random.choice(tstor_vals)
            total += extra
        
        results.append(total)
    
    return pd.DataFrame({"treach_yr": results})

def get_reach_transittimes(work_dir: str, river_name: str):
    """
    Processes all reach transit time distributions for a given river.
    
    Parameters:
        work_dir (str): Path to the working directory containing relevant data files.
        river_name (str): Name of the river to process.
    """
    # Path to transit length (storage) values
    nstor_path = os.path.join(work_dir, "RiverMapping", "Mobility", river_name, f"{river_name}_transit_lengths.csv")
    nstor_vals = pd.read_csv(nstor_path)
    
    # Path where Tstor distribution files are stored
    tstor_dir = os.path.join(work_dir, "RiverMapping", "Mobility", river_name, "Tstor_distributions")
    tstor_files = {
        file: pd.read_csv(os.path.join(tstor_dir, file))
        for file in os.listdir(tstor_dir)
        if file.endswith(".csv") and file.startswith("Reach_")
    }
    
    # Prepare output directory
    output_dir = os.path.join(work_dir, "RiverMapping", "Mobility", river_name, "treach_distributions")
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each reach
    for filename, tstor_df in tstor_files.items():
        try:
            reach_number = int(filename.split("_")[1])
            reach_transit_time_df = monte_carlo_reach_transit_time(tstor_df, nstor_vals, reach_number)
            output_path = os.path.join(output_dir, f"Reach_{reach_number}_treach_distribution.csv")
            reach_transit_time_df.to_csv(output_path, index=False)
            print(f"Saved: {output_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

## Calculate distributions for total alluvial transit time (t_tot)

In [6]:
def calculate_ttot_statistics(directory: str):
    """
    Calculates and saves statistics for all total transit time distribution CSV files
    found in the given directory.

    Parameters:
        directory (str): Directory containing ttot distribution CSV files.
    """
    files = [f for f in os.listdir(directory) if f.endswith("_distribution.csv")]

    for file in files:
        file_path = os.path.join(directory, file)
        ttt_df = pd.read_csv(file_path)

        if "ttot_yr" not in ttt_df.columns:
            print(f"Skipping {file} — missing 'ttot_yr' column.")
            continue

        # Compute statistics for all columns
        stats_list = []
        for column in ttt_df.columns:
            stats_list.append({
                "Variable": column,
                "Mean": np.mean(ttt_df[column]),
                "Standard Deviation": np.std(ttt_df[column]),
                "Min": np.min(ttt_df[column]),
                "1st Quartile": np.percentile(ttt_df[column], 25),
                "Median": np.median(ttt_df[column]),
                "3rd Quartile": np.percentile(ttt_df[column], 75),
                "Max": np.max(ttt_df[column])
            })

        stats_df = pd.DataFrame(stats_list)

        # Build output file name
        base_name = os.path.splitext(file)[0]
        stats_file = f"{base_name}_stats.csv"
        stats_path = os.path.join(directory, stats_file)
        stats_df.to_csv(stats_path, index=False)
        print(f"Saved stats: {stats_path}")

def get_total_transit_times(working_dir: str, river_name: str, num_iterations: int = 10_000, reach_start: int = 1, reach_end: int = None):
    """
    Runs a Monte Carlo simulation to compute the total river transit time distribution,
    and includes the sampled reach-level transit times for each iteration.
    
    Parameters:
        working_dir (str): Root directory containing the data folder structure.
        river_name (str): Name of the river for output file naming.
        num_iterations (int): Number of iterations for the Monte Carlo simulation (default is 10,000).
        reach_start (int): Index of the first reach to include (1-based, inclusive).
        reach_end (int): Index of the last reach to include (1-based, inclusive). If None, includes all reaches to the end.
    
    Returns:
        pd.DataFrame: DataFrame containing the total river transit time and individual reach samples.
    """
    rtt_dir = os.path.join(working_dir, 'RiverMapping', 'Mobility', river_name, 'treach_distributions')
    all_rtt_files = os.listdir(rtt_dir)
    
    # Determine reach range
    reach_end = reach_end if reach_end is not None else 100
    
    selected_reach_dfs = []
    actual_reaches = []
    
    for reach_num in range(reach_start, reach_end + 1):
        expected_filename = f"Reach_{reach_num}_treach_distribution.csv"
        file_path = os.path.join(rtt_dir, expected_filename)
        
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if "treach_yr" not in df.columns:
                raise KeyError(f"Missing 'treach_yr' column in file: {expected_filename}")
            selected_reach_dfs.append(df)
            actual_reaches.append(reach_num)
        else:
            raise FileNotFoundError(f"Expected file not found: {expected_filename}")
    
    simulation_results = []
    
    for _ in range(num_iterations):
        sampled_reach_times = [np.random.choice(df["treach_yr"], 1)[0] for df in selected_reach_dfs]
        total_time = sum(sampled_reach_times)
        simulation_results.append(sampled_reach_times + [total_time])
    
    # Build DataFrame with individual reach samples and total time
    columns = [f"reach_{reach}_tt_yr" for reach in actual_reaches] + ["ttot_yr"]
    simulation_df = pd.DataFrame(simulation_results, columns=columns)
    
    # Create output filename reflecting reach range
    reach_range_str = f"R{reach_start}toR{reach_end}"
    output_filename = f"{river_name}_{reach_range_str}_ttot_distribution.csv"
    output_path = os.path.join(working_dir, 'RiverMapping', 'Mobility', river_name, output_filename)
    
    simulation_df.to_csv(output_path, index=False)
    print(f"Saved: {output_path}")
    
    stats_directory = os.path.join(working_dir, 'RiverMapping', 'Mobility', river_name)
    calculate_ttot_statistics(stats_directory)

In [7]:
csv_path = r"E:\Dissertation\Data\GreenbergZhaoRees_river_datasheet.csv"
working_directory = r"E:\Dissertation\Data"
river_name = "Yukon_Beaver"
iterations = 10000
reach_start = 1
reach_end = 1

In [8]:
get_mobility(csv_path)

Processing Araguaia_LuizAlves Reach 1...
Saved corrected A_w totals for Reach 1 to E:\Dissertation\Data/RiverMapping/Mobility/Araguaia_LuizAlves\AW_distributions\Reach_1_aw_dist.csv


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Saved mobility plots to E:\Dissertation\Data/RiverMapping/Mobility/Araguaia_LuizAlves\Mobility_plots\Reach_1_mobility_fits.png
Saved mobility metrics for Araguaia_LuizAlves to E:\Dissertation\Data/RiverMapping/Mobility/Araguaia_LuizAlves\Araguaia_LuizAlves_mobility_metrics.csv
Processing Araguaia_SaoFelixDoAraguaia Reach 1...
Saved corrected A_w totals for Reach 1 to E:\Dissertation\Data/RiverMapping/Mobility/Araguaia_SaoFelixDoAraguaia\AW_distributions\Reach_1_aw_dist.csv


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Saved mobility plots to E:\Dissertation\Data/RiverMapping/Mobility/Araguaia_SaoFelixDoAraguaia\Mobility_plots\Reach_1_mobility_fits.png
Saved mobility metrics for Araguaia_SaoFelixDoAraguaia to E:\Dissertation\Data/RiverMapping/Mobility/Araguaia_SaoFelixDoAraguaia\Araguaia_SaoFelixDoAraguaia_mobility_metrics.csv
Processing Beas_MandiPlain Reach 1...
Saved corrected A_w totals for Reach 1 to E:\Dissertation\Data/RiverMapping/Mobility/Beas_MandiPlain\AW_distributions\Reach_1_aw_dist.csv


...

Saved mobility plots to E:\Dissertation\Data/RiverMapping/Mobility/Beni\Mobility_plots\Reach_10_mobility_fits.png
Saved mobility metrics for Beni to E:\Dissertation\Data/RiverMapping/Mobility/Beni\Beni_mobility_metrics.csv
Processing Yukon_Beaver Reach 1...
Saved corrected A_w totals for Reach 1 to E:\Dissertation\Data/RiverMapping/Mobility/Yukon_Beaver\AW_distributions\Reach_1_aw_dist.csv


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Saved mobility plots to E:\Dissertation\Data/RiverMapping/Mobility/Yukon_Beaver\Mobility_plots\Reach_1_mobility_fits.png
Saved mobility metrics for Yukon_Beaver to E:\Dissertation\Data/RiverMapping/Mobility/Yukon_Beaver\Yukon_Beaver_mobility_metrics.csv
Processing Koyukuk_Huslia Reach 1...
Saved corrected A_w totals for Reach 1 to E:\Dissertation\Data/RiverMapping/Mobility/Koyukuk_Huslia\AW_distributions\Reach_1_aw_dist.csv


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Saved mobility plots to E:\Dissertation\Data/RiverMapping/Mobility/Koyukuk_Huslia\Mobility_plots\Reach_1_mobility_fits.png
Saved mobility metrics for Koyukuk_Huslia to E:\Dissertation\Data/RiverMapping/Mobility/Koyukuk_Huslia\Koyukuk_Huslia_mobility_metrics.csv


In [ ]:
get_tstor_distributions(csv_path)

In [ ]:
get_reach_transittimes(working_directory, river_name)

In [ ]:
get_total_transit_times(working_directory, river_name, iterations, reach_start, reach_end)

## This checks what reaches have successfully calculated mobility metrics
As well as the number of masks that exist for each reach

In [ ]:
import pandas as pd
import os
from pathlib import Path

def check_reach_diagnostics(input_csv_path, output_csv_path):
    """
    Check mask counts and mobility CSV existence for each reach in the dataset.
    
    Parameters:
        input_csv_path (str): Path to the Greenberg et al. 2024 river datasheet CSV
        output_csv_path (str): Path to save the diagnostic output CSV
    
    Outputs:
        CSV file with columns:
            - river_name: Name of the river
            - ds_order: Reach number
            - num_masks: Count of .tif files in the Cleaned folder
            - mobility_csv_exists: 'yes' or 'no' indicating if mobility metrics CSV exists
    """
    # Read the input CSV
    river_data = pd.read_csv(input_csv_path)
    
    # List to store diagnostic results
    diagnostics = []
    
    # Iterate through each river
    for _, row in river_data.iterrows():
        river_name = row['river_name']
        working_directory = row['working_directory']
        
        print(f"Checking {river_name}...")
        
        # Define paths
        base_raster_dir = os.path.join(working_directory, "RiverMapping", "RiverMasks", river_name)
        mobility_csv_path = os.path.join(working_directory, "RiverMapping", "Mobility", river_name, 
                                        f"{river_name}_mobility_metrics.csv")
        
        # Check if mobility CSV exists (once per river, applies to all reaches)
        mobility_exists = "yes" if os.path.exists(mobility_csv_path) else "no"
        
        # Check if base raster directory exists
        if not os.path.exists(base_raster_dir):
            print(f"  Warning: Raster directory not found for {river_name}")
            diagnostics.append({
                'river_name': river_name,
                'ds_order': 'N/A',
                'num_masks': 0,
                'mobility_csv_exists': mobility_exists
            })
            continue
        
        # Find all reach directories
        try:
            reach_dirs = [d for d in os.listdir(base_raster_dir) 
                         if d.startswith("reach_") and os.path.isdir(os.path.join(base_raster_dir, d))]
            
            if not reach_dirs:
                print(f"  Warning: No reach directories found for {river_name}")
                diagnostics.append({
                    'river_name': river_name,
                    'ds_order': 'N/A',
                    'num_masks': 0,
                    'mobility_csv_exists': mobility_exists
                })
                continue
            
            # Sort reaches numerically
            reach_numbers = sorted([int(d.split('_')[1]) for d in reach_dirs])
            
            # Process each reach
            for ds_order in reach_numbers:
                cleaned_dir = os.path.join(base_raster_dir, f"reach_{ds_order}", "Cleaned")
                
                # Count .tif files
                if os.path.exists(cleaned_dir):
                    tif_files = [f for f in os.listdir(cleaned_dir) if f.endswith('.tif')]
                    num_masks = len(tif_files)
                else:
                    num_masks = 0
                    print(f"  Warning: Cleaned directory not found for reach {ds_order}")
                
                # Add to diagnostics
                diagnostics.append({
                    'river_name': river_name,
                    'ds_order': ds_order,
                    'num_masks': num_masks,
                    'mobility_csv_exists': mobility_exists
                })
                
                print(f"  Reach {ds_order}: {num_masks} masks, Mobility CSV: {mobility_exists}")
        
        except Exception as e:
            print(f"  Error processing {river_name}: {e}")
            diagnostics.append({
                'river_name': river_name,
                'ds_order': 'ERROR',
                'num_masks': 0,
                'mobility_csv_exists': mobility_exists
            })
    
    # Create DataFrame and save to CSV
    diagnostics_df = pd.DataFrame(diagnostics)
    diagnostics_df.to_csv(output_csv_path, index=False)
    
    print(f"\nDiagnostic results saved to {output_csv_path}")
    print(f"Total reaches checked: {len(diagnostics_df)}")
    print(f"Reaches with masks: {len(diagnostics_df[diagnostics_df['num_masks'] > 0])}")
    print(f"Rivers with mobility CSV: {diagnostics_df['mobility_csv_exists'].value_counts().get('yes', 0)} yes, {diagnostics_df['mobility_csv_exists'].value_counts().get('no', 0)} no")
    
    return diagnostics_df


# Example usage
if __name__ == "__main__":
    input_csv = r"E:\Dissertation\Data\Zhaoetal2025_river_datasheet.csv"
    output_csv = r"C:\Users\huckr\Desktop\UCSB\Dissertation\Data\Code\Troubleshooting\reach_mobility_calcs_diagnostics.csv"
    
    results = check_reach_diagnostics(input_csv, output_csv)
    
    # Display summary statistics
    print("\n=== SUMMARY STATISTICS ===")
    print(f"Mean masks per reach: {results['num_masks'].mean():.2f}")
    print(f"Median masks per reach: {results['num_masks'].median():.0f}")
    print(f"Min masks: {results['num_masks'].min()}")
    print(f"Max masks: {results['num_masks'].max()}")
    print(f"\nReaches with < 5 masks:")
    low_mask_reaches = results[results['num_masks'] < 5]
    if len(low_mask_reaches) > 0:
        for _, row in low_mask_reaches.iterrows():
            print(f"  {row['river_name']} Reach {row['ds_order']}: {row['num_masks']} masks")
    else:
        print("  None")